In [39]:
import csv
import json
import os

In [40]:
# transform the Gemini output csv to json files that can be used in existing pipeline
model = "gemini-2.5-flash-lite"
model_short = "gemini"
dataset = "cub"
filename_appendix = "_08-09"
input_file = os.path.join("outputs", "{}_{}_concepts{}.csv".format(model, dataset, filename_appendix))
output_file = os.path.join(os.path.dirname(os.getcwd()), "data/concept_sets/{}_init/{}_{}_{}.json")

values = []
important = {}
superclass = {}
around = {}
wrapper_list = [important, superclass, around]
wrapper_names = ["important", "superclass", "around"]

with open(input_file, "r") as f:
    reader = csv.reader(f)
    i = 0
    for line in reader:
        #print("new line", line[2])
        class_name = line[0]
        concepts = line[2].splitlines()
        concepts = [c.replace("*", "") for c in concepts]
        for default_concept in concepts:
            concept = default_concept.replace("*", "").strip()
            if len(concept) > 0 and concept[0] == "-":
                concept = concept[1:].strip()
            if len(concept) == 0:
               continue
            print("start_concept: ", default_concept, "  ###  processed_concept: ", concept)
            values.append(concept)
        wrapper_list[i][class_name] = values
        i += 1
        if i == len(wrapper_list):
            i = 0

start_concept:  Here are the most important features for recognizing a "Black_footed_Albatross":   ###  processed_concept:  Here are the most important features for recognizing a "Black_footed_Albatross":
start_concept:  -   Dark, sooty-brown plumage: The entire body, wings, and head are uniformly dark brown.   ###  processed_concept:  Dark, sooty-brown plumage: The entire body, wings, and head are uniformly dark brown.
start_concept:  -   Large size: A big seabird with a substantial wingspan.   ###  processed_concept:  Large size: A big seabird with a substantial wingspan.
start_concept:  -   Long, narrow wings: Adapted for long-distance gliding over the ocean.   ###  processed_concept:  Long, narrow wings: Adapted for long-distance gliding over the ocean.
start_concept:  -   Hooked beak: A characteristic feature, often black with a pale tip.   ###  processed_concept:  Hooked beak: A characteristic feature, often black with a pale tip.
start_concept:  -   Pale feet: The feet and legs 

In [41]:
for i, feature_dict in enumerate(wrapper_list):
    json_object = json.dumps(feature_dict, indent=4)
    if not os.path.exists(os.path.dirname(output_file.format(model_short, model_short, dataset, wrapper_names[i]))):
        os.makedirs(os.path.dirname(output_file.format(model_short, model_short, dataset, wrapper_names[i])))
    with open(output_file.format(model_short, model_short, dataset, wrapper_names[i]), "w") as outfile:
        outfile.write(json_object)